https://huggingface.co/docs/transformers/tasks/sequence_classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets torch sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading the model:

In [4]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    pipeline
)
from datasets import load_dataset, load_metric
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
model_name = "dbauer1860/deberta-v3-base-fake-hotel-review-detector"

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Ott et al. Dataset

In [9]:
ott_dataset = load_dataset(
    "csv",
    data_files= "/content/drive/MyDrive/data/ott_et_al_processed.csv",
    delimiter=";"
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
ott_dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 1600
})

In [11]:
predictions = classifier(ott_dataset['train']['text'], max_length=512, truncation=True)

In [12]:
for prediction in predictions:
    print(f"Label: {prediction['label']}, Score: {prediction['score']}")

Label: FAKE, Score: 0.9982666969299316
Label: REAL, Score: 0.9997897744178772
Label: REAL, Score: 0.9997449517250061
Label: REAL, Score: 0.9997830986976624
Label: REAL, Score: 0.9997311234474182
Label: REAL, Score: 0.9997609257698059
Label: REAL, Score: 0.9997344613075256
Label: REAL, Score: 0.9997519850730896
Label: REAL, Score: 0.9997163414955139
Label: REAL, Score: 0.999761164188385
Label: REAL, Score: 0.9997095465660095
Label: REAL, Score: 0.9997749924659729
Label: REAL, Score: 0.9995903372764587
Label: REAL, Score: 0.999646782875061
Label: REAL, Score: 0.9996730089187622
Label: REAL, Score: 0.9997873902320862
Label: REAL, Score: 0.9994571805000305
Label: REAL, Score: 0.9996687173843384
Label: REAL, Score: 0.999744713306427
Label: REAL, Score: 0.9997960925102234
Label: REAL, Score: 0.99964439868927
Label: REAL, Score: 0.999256432056427
Label: REAL, Score: 0.9997636675834656
Label: REAL, Score: 0.9997404217720032
Label: REAL, Score: 0.9997486472129822
Label: REAL, Score: 0.999680638

In [13]:
predicted_labels = [prediction['label'] for prediction in predictions]
scores = [prediction['score'] for prediction in predictions]

In [15]:
label_dict = {'FAKE': 0, 'REAL': 1}
predicted_labels_numeric = [label_dict[label] for label in predicted_labels]
true_labels = ott_dataset['train']['label']

In [16]:
accuracy = accuracy_score(true_labels, predicted_labels_numeric)
precision = precision_score(true_labels, predicted_labels_numeric, average='binary')
recall = recall_score(true_labels, predicted_labels_numeric, average='binary')
f1 = f1_score(true_labels, predicted_labels_numeric, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.505
Precision: 0.5025412960609911
Recall: 0.98875
F1 Score: 0.6663858466722831
